In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings('ignore')

# Thiết lập hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Đã import các thư viện thành công!")


Đã import các thư viện thành công!


In [41]:
# Load models và dữ liệu
print("=" * 60)
print("ĐANG LOAD MODELS VÀ DỮ LIỆU...")
print("=" * 60)

# Load SVD model
with open('../models/svd_model.pkl', 'rb') as f:
    svd = pickle.load(f)

# Load TF-IDF vectorizer
with open('../models/tfidf_vectorizer.pkl', 'rb') as f:
    tfidf = pickle.load(f)

# Load factors
user_factors = np.load('../models/user_factors.npy')
item_factors = np.load('../models/item_factors.npy')

# Load mappings
with open('../models/movie_id_to_idx.pkl', 'rb') as f:
    movie_id_to_idx = pickle.load(f)

with open('../models/user_id_to_idx.pkl', 'rb') as f:
    user_id_to_idx = pickle.load(f)

with open('../models/tfidf_movie_id_to_row.pkl', 'rb') as f:
    tfidf_movie_id_to_row = pickle.load(f)

# Load dataframes
movies_df_clean = pd.read_pickle('../models/movies_df_clean.pkl')
train_df = pd.read_pickle('../models/train_df.pkl')
tfidf_df = pd.read_pickle('../models/tfidf_df.pkl')

# Tạo TF-IDF matrix từ tfidf_df
tfidf_matrix = csr_matrix(tfidf_df.values)

print(f"\n✓ Đã load thành công!")
print(f"  - User factors shape: {user_factors.shape}")
print(f"  - Item factors shape: {item_factors.shape}")
print(f"  - TF-IDF matrix shape: {tfidf_matrix.shape}")
print(f"  - Train ratings: {len(train_df):,}")


ĐANG LOAD MODELS VÀ DỮ LIỆU...

✓ Đã load thành công!
  - User factors shape: (668, 50)
  - Item factors shape: (3854, 50)
  - TF-IDF matrix shape: (3855, 21)
  - Train ratings: 75,296


In [42]:
# Load dữ liệu gốc và tạo test set (giống như trong train_models.py)
print("=" * 60)
print("TẠO TEST SET...")
print("=" * 60)

# Load dữ liệu gốc
ratings_df = pd.read_csv('../data/ratings.csv')

# Làm sạch dữ liệu (giống train_models.py)
ratings_df_clean = ratings_df.dropna(subset=['userId', 'movieId', 'rating'])
ratings_df_clean = ratings_df_clean.drop_duplicates(subset=['userId', 'movieId'], keep='last')
ratings_df_clean = ratings_df_clean[
    (ratings_df_clean['rating'] >= 0.5) & 
    (ratings_df_clean['rating'] <= 5.0)
]

# Chỉ giữ các phim có trong movies_df_clean
ratings_df_clean = ratings_df_clean[ratings_df_clean['movieId'].isin(movies_df_clean['movieId'])]

# Chia train/test với cùng random_state=42
train_df_new, test_df = train_test_split(
    ratings_df_clean,
    test_size=0.2,
    random_state=42
)

print(f"Train set: {len(train_df_new):,} ratings")
print(f"Test set: {len(test_df):,} ratings")
print(f"\n✓ Đã tạo test set thành công!")


TẠO TEST SET...
Train set: 75,296 ratings
Test set: 18,825 ratings

✓ Đã tạo test set thành công!


In [43]:
# Global mean
global_mean = train_df["rating"].mean()

# User bias
user_bias = (
    train_df.groupby("userId")["rating"].mean() - global_mean
).to_dict()

# Movie bias
movie_bias = (
    train_df.groupby("movieId")["rating"].mean() - global_mean
).to_dict()


In [44]:
# Hàm dự đoán Collaborative Filtering
def predict_rating_cf(user_id,movie_id,user_factors,item_factors,user_id_to_idx,movie_id_to_idx,global_mean,user_bias, movie_bias
):
    """
    CF prediction với bias:
    r̂_ui = μ + b_u + b_i + p_u^T q_i
    """

    # Bias fallback
    bu = user_bias.get(user_id, 0.0)
    bi = movie_bias.get(movie_id, 0.0)

    # Nếu user hoặc movie chưa thấy → chỉ dùng bias
    if user_id not in user_id_to_idx or movie_id not in movie_id_to_idx:
        return global_mean + bu + bi

    uidx = user_id_to_idx[user_id]
    midx = movie_id_to_idx[movie_id]

    interaction = np.dot(user_factors[uidx], item_factors[midx])

    pred = global_mean + bu + bi + interaction
    return pred  # KHÔNG clip ở đây


# Hàm dự đoán Content-Based Filtering
def predict_rating_cb(
    user_id,
    movie_id,
    train_df,
    tfidf_matrix,
    tfidf_movie_id_to_row,
    global_mean,
    user_bias,
    movie_bias
):
    user_ratings = train_df.loc[
        train_df["userId"] == user_id, ["movieId", "rating"]
    ]

    # fallback = mean + bias
    base = (
        global_mean
        + user_bias.get(user_id, 0.0)
        + movie_bias.get(movie_id, 0.0)
    )

    if user_ratings.empty:
        return base

    target_row = tfidf_movie_id_to_row.get(int(movie_id))
    if target_row is None:
        return base

    rated = user_ratings.copy()
    rated["row"] = rated["movieId"].map(tfidf_movie_id_to_row)
    rated = rated.dropna(subset=["row"])
    if rated.empty:
        return base

    rated_rows = rated["row"].astype(int).to_numpy()
    ratings = rated["rating"].to_numpy(dtype=float)

    sims = (tfidf_matrix[rated_rows] @ tfidf_matrix[target_row].T).toarray().ravel()
    sim_sum = np.abs(sims).sum()

    if sim_sum == 0:
        return base

    return float((sims * ratings).sum() / sim_sum)


# Hàm dự đoán Hybrid
def predict_rating_hybrid(
    user_id,
    movie_id,
    user_factors,
    item_factors,
    user_id_to_idx,
    movie_id_to_idx,
    train_df,
    tfidf_matrix,
    tfidf_movie_id_to_row,
    global_mean,
    user_bias,
    movie_bias,
    cf_weight=0.3,
    cb_weight=0.7
):
    cf_pred = predict_rating_cf(
        user_id,
        movie_id,
        user_factors,
        item_factors,
        user_id_to_idx,
        movie_id_to_idx,
        global_mean,
        user_bias,
        movie_bias
    )

    cb_pred = predict_rating_cb(
        user_id,
        movie_id,
        train_df,
        tfidf_matrix,
        tfidf_movie_id_to_row,
        global_mean,
        user_bias,
        movie_bias
    )

    return cf_weight * cf_pred + cb_weight * cb_pred



In [45]:
# Đánh giá trên test set
print("=" * 60)
print("ĐANG ĐÁNH GIÁ TRÊN TEST SET...")
print("=" * 60)
print(f"Số lượng samples trong test set: {len(test_df):,}")
print("Đang tính toán predictions...")

# Lấy sample để test nhanh (có thể comment để chạy full test set)
# test_df_sample = test_df.sample(n=min(1000, len(test_df)), random_state=42)
test_df_sample = test_df  # Chạy full test set

# Dự đoán cho từng phương pháp
predictions_cf = []
predictions_cb = []
predictions_hybrid = []
actual = []

for _, row in test_df.iterrows():
    u = row["userId"]
    m = row["movieId"]
    r = row["rating"]

    predictions_cf.append(
        predict_rating_cf(
            u, m,
            user_factors, item_factors,
            user_id_to_idx, movie_id_to_idx,
            global_mean, user_bias, movie_bias
        )
    )

    predictions_cb.append(
        predict_rating_cb(
            u, m,
            train_df,
            tfidf_matrix, tfidf_movie_id_to_row,
            global_mean, user_bias, movie_bias
        )
    )

    predictions_hybrid.append(
        predict_rating_hybrid(
            u, m,
            user_factors, item_factors,
            user_id_to_idx, movie_id_to_idx,
            train_df,
            tfidf_matrix, tfidf_movie_id_to_row,
            global_mean, user_bias, movie_bias
        )
    )

    actual.append(r)

actual = np.array(actual)
predictions_cf = np.array(predictions_cf)
predictions_cb = np.array(predictions_cb)
predictions_hybrid = np.array(predictions_hybrid)


print(f"\n✓ Đã hoàn thành dự đoán cho {len(predictions_cf):,} samples!")


ĐANG ĐÁNH GIÁ TRÊN TEST SET...
Số lượng samples trong test set: 18,825
Đang tính toán predictions...

✓ Đã hoàn thành dự đoán cho 18,825 samples!


In [46]:
results_df = pd.DataFrame({
    "Model": ["Collaborative Filtering", "Content-Based", "Hybrid"],
    "RMSE": [
        np.sqrt(mean_squared_error(actual, predictions_cf)),
        np.sqrt(mean_squared_error(actual, predictions_cb)),
        np.sqrt(mean_squared_error(actual, predictions_hybrid))
    ],
    "MAE": [
        mean_absolute_error(actual, predictions_cf),
        mean_absolute_error(actual, predictions_cb),
        mean_absolute_error(actual, predictions_hybrid)
    ]
})

print(results_df)



                     Model      RMSE       MAE
0  Collaborative Filtering  1.279442  0.983108
1            Content-Based  0.913037  0.705797
2                   Hybrid  0.892313  0.677807


In [53]:
# Tính Precision@K và Recall@K
print("=" * 60)
print("TÍNH PRECISION@K VÀ RECALL@K...")
print("=" * 60)

# Ngưỡng để coi là "relevant" (rating >= threshold)
threshold = 3.5

# Tạo DataFrame với predictions và actual ratings
test_results = test_df.copy()
test_results['actual'] = actual
test_results['pred_cf'] = predictions_cf
test_results['pred_cb'] = predictions_cb
test_results['pred_hybrid'] = predictions_hybrid

# Đánh dấu relevant items (actual rating >= threshold)
test_results['is_relevant'] = (test_results['actual'] >= threshold).astype(int)

# Các giá trị K cần tính
K_values = [5, 10, 20]

def calculate_precision_recall_at_k(test_results, pred_col, K_values, threshold=3.5):
    """
    Tính Precision@K và Recall@K cho một model
    
    Args:
        test_results: DataFrame với columns ['userId', 'movieId', 'actual', pred_col, 'is_relevant']
        pred_col: Tên column chứa predictions
        K_values: List các giá trị K cần tính
        threshold: Ngưỡng để coi là relevant
    
    Returns:
        Dict với keys là K values, values là (precision@k, recall@k)
    """
    results = {}
    
    for K in K_values:
        precisions = []
        recalls = []
        
        # Nhóm theo user
        for user_id, user_data in test_results.groupby('userId'):
            # Sắp xếp theo predicted rating giảm dần
            user_data_sorted = user_data.sort_values(pred_col, ascending=False)
            
            # Lấy top-K
            top_k = user_data_sorted.head(K)
            
            # Số relevant items trong top-K
            relevant_in_topk = top_k['is_relevant'].sum()
            
            # Tổng số relevant items của user
            total_relevant = user_data['is_relevant'].sum()
            
            # Tính Precision@K
            if len(top_k) > 0:
                precision_k = relevant_in_topk / len(top_k)
            else:
                precision_k = 0.0
            
            # Tính Recall@K
            if total_relevant > 0:
                recall_k = relevant_in_topk / total_relevant
            else:
                recall_k = 0.0  # Nếu user không có relevant items, recall = 0
            
            precisions.append(precision_k)
            recalls.append(recall_k)
        
        # Tính trung bình qua tất cả users
        avg_precision = np.mean(precisions) if precisions else 0.0
        avg_recall = np.mean(recalls) if recalls else 0.0
        
        results[K] = (avg_precision, avg_recall)
    
    return results

# Tính cho từng model
print(f"\nNgưỡng rating để coi là 'relevant': >= {threshold}")
print(f"\nĐang tính toán Precision@K và Recall@K...")

results_cf = calculate_precision_recall_at_k(test_results, 'pred_cf', K_values, threshold)
results_cb = calculate_precision_recall_at_k(test_results, 'pred_cb', K_values, threshold)
results_hybrid = calculate_precision_recall_at_k(test_results, 'pred_hybrid', K_values, threshold)

# Tạo DataFrame để hiển thị kết quả
precision_recall_data = []

for K in K_values:
    prec_cf, rec_cf = results_cf[K]
    prec_cb, rec_cb = results_cb[K]
    prec_hybrid, rec_hybrid = results_hybrid[K]
    
    precision_recall_data.append({
        'K': K,
        'CF_Precision@K': prec_cf,
        'CF_Recall@K': rec_cf,
        'CB_Precision@K': prec_cb,
        'CB_Recall@K': rec_cb,
        'Hybrid_Precision@K': prec_hybrid,
        'Hybrid_Recall@K': rec_hybrid
    })

precision_recall_df = pd.DataFrame(precision_recall_data)

print("\n" + "=" * 80)
print("KẾT QUẢ PRECISION@K VÀ RECALL@K:")
print("=" * 80)
print(precision_recall_df.to_string(index=False))

# Thêm vào results_df tổng hợp (ví dụ với K=10)
if 'Precision@10' not in results_df.columns:
    results_df['Precision@10'] = [
        results_cf[10][0],
        results_cb[10][0],
        results_hybrid[10][0]
    ]
    results_df['Recall@10'] = [
        results_cf[10][1],
        results_cb[10][1],
        results_hybrid[10][1]
    ]

print("\n" + "=" * 80)
print("KẾT QUẢ TỔNG HỢP (với K=10):")
print("=" * 80)
print(results_df.to_string(index=False))

print(f"\n✓ Đã tính Precision@K và Recall@K thành công!")


TÍNH PRECISION@K VÀ RECALL@K...

Ngưỡng rating để coi là 'relevant': >= 3.5

Đang tính toán Precision@K và Recall@K...

KẾT QUẢ PRECISION@K VÀ RECALL@K:
 K  CF_Precision@K  CF_Recall@K  CB_Precision@K  CB_Recall@K  Hybrid_Precision@K  Hybrid_Recall@K
 5        0.755997     0.481790        0.702024     0.455707            0.758696         0.482514
10        0.714026     0.671375        0.692137     0.660981            0.721822         0.678936
20        0.683333     0.822682        0.672239     0.814736            0.686556         0.824981

KẾT QUẢ TỔNG HỢP (với K=10):
                  Model     RMSE      MAE  Precision   Recall  Precision@10  Recall@10
Collaborative Filtering 1.279442 0.983108   0.731811 0.826758      0.714026   0.671375
          Content-Based 0.913037 0.705797   0.769940 0.682769      0.692137   0.660981
                 Hybrid 0.892313 0.677807   0.760361 0.794976      0.721822   0.678936

✓ Đã tính Precision@K và Recall@K thành công!
